# SigMA on Flame Nebula

The idea of this notebook is to run SigMA on the 4D space we have available from VISIONS. Wish me luck!
### Modules

In [1]:
# Python modules
import os
from typing import Union
import pandas as pd
import numpy as np
from astropy.coordinates import LSR, SkyCoord, Distance
import astropy.units as u

#### DistantSigMA modules
from DistantSigMA.DistantSigMA.cluster_simulations import calculate_std_devs
from DistantSigMA.DistantSigMA.scalefactor_sampling import lhc_lloyd
from DistantSigMA.misc import utilities as ut
from DistantSigMA.DistantSigMA.setup_and_scaling import parameter_scaler, single_parameter_scaling, save_output_summary
from SigMA.SigMA import SigMA
from DistantSigMA.DistantSigMA.clustering_routine import remove_field_stars, extract_signal_remove_spurious, extract_signal, consensus_function
from coordinate_transformations.sky_convert import gal_vtan_lsr

## Setup

In [2]:
# Paths
output_path = ut.set_output_path(script_name="4D_clustering")

run = "first_try"
output_path = output_path + f"{run}/"
if not os.path.exists(output_path):
    os.makedirs(output_path)

## Data

*Note: The input file needs to contain the following columns: ['ra', 'dec', 'parallax', 'pmra', 'pmdec']*

In [3]:
# load data for error sampling (already slimmed)
error_sampling_df = pd.read_csv("../../Data/Gaia/Gaia_DR3_500pc_10percent.csv")

### IR df

In [4]:
IR_new = ['ra', 'dec', 'parallax', 'pmra', 'pmdec']
IR_old = ["mean_RA", "mean_DEC", "pmra_IR", "pmde_IR"]
ir_dict = dict(zip(IR_old, IR_new))

df_IR = pd.read_csv('../../Data/tmp/master-lite_x_Spitzer_Megeath_Gaia_Nemesis.csv', usecols=("mean_RA", "mean_DEC", "pmra_IR", "pmde_IR", "RAdeg", "DEdeg", "Plx", "pmRA", "pmDE",
                                                                                                "e_RAdeg", "e_DEdeg", "e_Plx", "e_pmRA", "e_pmDE"))

df_IR.rename(columns=ir_dict, inplace=True)

### Gaia df

In [5]:
gaia_old = ["RAdeg", "DEdeg", "Plx", "pmRA", "pmDE", "e_RAdeg", "e_DEdeg", "e_Plx", "e_pmRA", "e_pmDE"]
gaia_new = ['ra', 'dec', 'parallax', 'pmra', 'pmdec', "ra_error", "dec_error", "parallax_error", "pmra_error", "pmde_error"]
gaia_dict = dict(zip(gaia_old, gaia_new))

# load the dataframe - need 2 for this because of the naming conventions
df_gaia = pd.read_csv('../../Data/tmp/master-lite_x_Spitzer_Megeath_Gaia_Nemesis.csv', usecols=("RAdeg", "DEdeg", "Plx", "pmRA", "pmDE",
                                                                                                "e_RAdeg", "e_DEdeg", "e_Plx", "e_pmRA", "e_pmDE"))

df_gaia.rename(columns=gaia_dict, inplace=True)

### Add cartesian coordinates

In [6]:
skycoord = SkyCoord(
        ra=df_gaia["ra"].to_numpy() * u.deg,
        dec=df_gaia["dec"].to_numpy() * u.deg,  # 2D on sky postition
        distance=Distance(parallax=df_gaia["parallax"].to_numpy() * u.mas, allow_negative=True),  # distance in pc
        pm_ra_cosdec=df_gaia["pmra"].to_numpy() * u.mas / u.yr,
        pm_dec=df_gaia["pmdec"].to_numpy() * u.mas / u.yr,
        radial_velocity=0.0 * u.km / u.s,
        frame="icrs",
    )

df = gal_vtan_lsr(skycoord)
df_chunk = pd.concat([df_gaia, df], axis=1)
df_chunk.dropna(subset=["ra", "dec", "parallax", "pmra", "pmdec"], inplace=True)


### constrain the Distance

In [7]:
df_chunk["D"] = 1000/df_chunk["parallax"]

df_small = df_chunk[(df_chunk.D > 10) & (df_chunk.D < 1000)]

In [8]:
# set variables manually as there is no coarse segmentation in the input dataset right now
chunk = 0
result_path = output_path

In [9]:
df_small.head()

,ra,dec,ra_error,dec_error,parallax,parallax_error,pmra,pmra_error,pmdec,pmde_error,X,Y,Z,v_a_lsr,v_d_lsr,D
0,85.551753,-2.086052,0.0237,0.0214,2.5119,0.0274,1.424,0.028,0.330,0.024,-341.124422,-172.016934,-111.946015,2.251009,7.092503,398.105020
1,85.386485,-1.965999,0.0220,0.0207,2.6756,0.0285,2.093,0.026,-1.232,0.024,-320.641056,-160.350540,-105.662402,3.224025,4.322255,373.747944
3,85.260554,-1.727195,1.1237,1.0010,3.6669,1.4756,-1.335,1.221,0.369,1.042,-234.531613,-115.851212,-77.098112,-2.246544,7.052169,272.709918
5,85.321513,-1.731072,0.2782,0.2881,2.4825,0.4087,0.377,0.322,-0.389,0.322,-346.419964,-171.370401,-113.530330,0.216861,5.831167,402.819738
7,85.326645,-1.831449,0.1863,0.1996,2.7434,0.3040,-1.510,0.257,-0.343,0.231,-313.149436,-155.558950,-102.991515,-3.110767,5.951858,364.511190


## Clustering

Currently written for the scenario that the input is a single chunk without need for further splitting.

### A) Preliminary solution -- Gaia data

My idea is to make the preliminary cluster using the subset of the sources with Gaia data -- this way we should be able to determine the scaling factors nicely. *Maybe I will need to add more Gaia sources in the region if it does not work at first.*

In [10]:
def setup_4D(df_fit: pd.DataFrame, sf_range: Union[list, np.linspace, range],
                  KNN_list: Union[list, np.linspace, range], beta: float, knn_initcluster_graph: int,
                  scaling: str = None, means=None):

    # resampling is not implemented for ICRS yet
    n_resampling = 0

    # scale the ICRS parameters
    scaled_cols = ['ra', 'dec', 'pmra', 'pmdec']
    df_fin = df_fit
    scale_factors = means

    # SigMA kwargs
    sigma_kwargs = dict(cluster_features=scaled_cols, scale_factors=scale_factors, nb_resampling=n_resampling,
                        max_knn_density=max(KNN_list) + 1, beta=beta, knn_initcluster_graph=knn_initcluster_graph)

    setup_dict = {"scale_factor_list": sf_range, "scale_factors": scale_factors, "sigma_kwargs": sigma_kwargs}

    return setup_dict, df_fin

In [11]:
def setup_ICRS_ps(df_fit: pd.DataFrame, sf_params: Union[str, list], sf_range: Union[list, np.linspace, range],
                  KNN_list: Union[list, np.linspace, range], beta: float, knn_initcluster_graph: int,
                  scaling: str = None, means=None, cols_to_scale = None):

    n_resampling = 0

    # scale the ICRS parameters
    if type(scaling) == str:
        df_scaled = df_fit.copy()
        if cols_to_scale is None:
            cols_to_scale = ['ra', 'dec', 'parallax', 'pmra', 'pmdec']
        else:
            cols_to_scale = cols_to_scale
        scaled_cols = ['ra_scaled', 'dec_scaled', 'parallax_scaled', 'pmra_scaled', 'pmdec_scaled']
        scaled_data = [parameter_scaler(df_scaled[col], scaling) for col in cols_to_scale]

        for col_id, col in enumerate(scaled_cols):
            df_scaled[col] = scaled_data[col_id]

        df_fin = df_scaled

    else:
        scaled_cols = cols_to_scale
        df_fin = df_fit

    if type(sf_params) == str:
        mean_sf, scale_factors = single_parameter_scaling(sf_params, sf_range)
    elif type(sf_params) == list:
        scale_factors = means

    # SigMA kwargs
    sigma_kwargs = dict(cluster_features=scaled_cols, scale_factors=scale_factors, nb_resampling=n_resampling,
                        max_knn_density=max(KNN_list) + 1, beta=beta, knn_initcluster_graph=knn_initcluster_graph)

    setup_dict = {"scale_factor_list": sf_range, "scale_factors": scale_factors, "sigma_kwargs": sigma_kwargs}

    return setup_dict, df_fin


In [12]:
def run_clustering(region_label, df_input, sf_params, parameter_dict: dict, mode: str, output_loc: str, setup_func,
                   column_means=None, cols_to_scale =None ):
    # most important variables
    KNNs = parameter_dict["KNN_list"]
    # setup kwargs
    if setup_func == "ICRS":
        setup_kwargs, df_focus = setup_ICRS_ps(df_fit=df_input, sf_params=sf_params, sf_range=parameter_dict["sfs"],
                                               KNN_list=KNNs, beta=parameter_dict["beta"],
                                               knn_initcluster_graph=parameter_dict["knn_initcluster_graph"],
                                               scaling=parameter_dict["scaling"], means=column_means, cols_to_scale=cols_to_scale)
    else:
        setup_kwargs, df_focus = setup_4D(df_fit = df_input, sf_range=parameter_dict["sfs"],  KNN_list=KNNs, beta=parameter_dict["beta"],
                                               knn_initcluster_graph=parameter_dict["knn_initcluster_graph"],
                                               scaling=parameter_dict["scaling"], means=column_means, )
    sigma_kwargs = setup_kwargs["sigma_kwargs"]
    scale_factor_list = setup_kwargs["scale_factor_list"]

    # ---------------------------------------------------------

    # initialize SigMA with sf_mean
    clusterer = SigMA(data=df_focus, **sigma_kwargs)
    # save X_mean
    X_mean_sf = clusterer.X
    # initialize array for density values (collect the rho_sums)
    rhosum_list = []

    # Initialize array for the outer cc (occ) results (remove field stars / rfs, remove spurious clusters / rsc)
    results_rfs = np.empty(shape=(len(KNNs), len(df_focus)))
    results_rsc = np.empty(shape=(len(KNNs), len(df_focus)))
    results_simple = np.empty(shape=(len(KNNs), len(df_focus)))

    # Outer loop: KNN
    for kid, knn in enumerate(KNNs):

        print(f"-- Current run with KNN = {knn} -- \n")

        label_matrix_rfs = np.empty(shape=(len(scale_factor_list), len(df_focus)))
        label_matrix_rsc = np.empty(shape=(len(scale_factor_list), len(df_focus)))
        label_matrix_simple = np.empty(shape=(len(scale_factor_list), len(df_focus)))

        # initialize density-sum over all scaling factors
        rho_sum = np.zeros(df_focus.shape[0], dtype=np.float32)

        # ---------------------------------------------------------
        df_labels = pd.DataFrame()
        # Inner loop: Scale factors
        for sf_id, sf in enumerate(scale_factor_list):
            # Set current scale factor
            if mode == "prelim":
                scale_factors = {'pos': {'features': ['parallax_scaled'], 'factor': sf}}
                clusterer.set_scaling_factors(scale_factors)
                print(f"Performing clustering for scale factor {clusterer.scale_factors['pos']['factor']}...")
            elif mode == "final":
                scale_factors = {'pos': {'features': ['ra', 'dec'], 'factor': list(sf[:3])},
                                 'vel': {'features': ['pmra', 'pmdec'], 'factor': list(sf[3:])}}
                clusterer.set_scaling_factors(scale_factors)
                print(f"Performing clustering for scale factor p{clusterer.scale_factors['pos']['factor']}"
                      f"{clusterer.scale_factors['vel']['factor']}...")

            # Fit
            clusterer.fit(alpha=parameter_dict["alpha"], knn=knn, bh_correction=parameter_dict["bh_correction"])
            label_array = clusterer.labels_

            # density and X
            rho, X = clusterer.weights_, clusterer.X
            rho_sum += rho

            # a) remove field stars
            nb_rfs = remove_field_stars(label_array, rho, label_matrix_rfs, sf_id)
            # b) remove spurious clusters
            nb_es, nb_rsc = extract_signal_remove_spurious(df_focus, label_array, rho, X, label_matrix_rsc, sf_id)
            # c) do new method
            nb_simple = extract_signal(label_array, clusterer, label_matrix_simple, sf_id)
            # Write the output to the hyperparameter file:

            save_output_summary(
                summary_str={"knn": knn, "sf": str(sf), "n_rfs": nb_rfs, "n_rsc": nb_rsc, "n_simple": nb_simple},
                file=output_loc + f"{mode}_ICC_{knn}_summary.csv")

            df_labels[f"#_{sf_id}"] = label_matrix_rsc[sf_id, :]

        if mode == "final":
            # save output files
            df_labels.to_csv(output_loc + f"{mode}_ICC_{knn}_labels.csv")
        # append the density sum to the list over all KNN
        rhosum_list.append(rho_sum)

        # Perform consensus clustering on the a) and b) arrays (automatically generates and saves a html-plot)
        labels_icc_rfs, n_icc_rfs = consensus_function(label_matrix_rfs, rho_sum, df_focus,
                                                       f"{mode}_Region_{int(region_label)}_rfs_KNN_{knn}_ICC",
                                                       output_loc,
                                                       plotting=True)
        labels_icc_rsc, n_icc_rsc = consensus_function(label_matrix_rsc, rho_sum, df_focus,
                                                       f"{mode}_Region_{int(region_label)}_rsc_KNN_{knn}_ICC",
                                                       output_loc,
                                                       plotting=True)

        labels_icc_simple, n_icc_simple = consensus_function(label_matrix_simple, rho_sum, df_focus,
                                                             f"{mode}_Region_{int(region_label)}_simple_KNN_{knn}_ICC",
                                                             output_loc,
                                                             plotting=True)
        results_rfs[kid, :] = labels_icc_rfs

        results_rsc[kid, :] = labels_icc_rsc
        results_simple[kid, :] = labels_icc_simple

        print(f":: Finished run for KNN={knn}! \n. Found {n_icc_rfs} / {n_icc_rsc} / {n_icc_simple} final clusters.")

    knn_mid = int(len(KNNs) / 2 - 1)
    df_save = df_focus.copy()
    label_lists = [results_rfs, results_rsc, results_simple]

    # Perform consensus clustering on the c) and d) steps
    labels_occ, n_occ = zip(
        *(consensus_function(jl, rhosum_list[knn_mid], df_focus, f"{mode}_Region_{int(region_label)}_{name}_OCC",
                             output_loc) for jl, name in zip(label_lists, ["rfs", "rsc", "simple"])))
    n_occ = list(n_occ)
    labels_occ = list(labels_occ)

    # save the labels in a csv file and plot the result
    df_save["rsc"] = labels_occ[1]
    df_save["rfs"] = labels_occ[0]
    df_save["simple"] = labels_occ[2]
    df_save.to_csv(output_loc + f"{mode}_Region_{int(region_label)}_results_CC.csv")

    save_output_summary(summary_str={"knn": "occ", "n_rfs": n_occ[0], "n_rsc": n_occ[1],
                                     "n_rfs_cleaned": n_occ[2]},
                        file=output_loc + f"{mode}_Region_{int(region_label)}_outer_summary.csv")

    # Output log-file
    filename = output_loc + f"Region_{region_label}_{mode}_parameters.txt"
    with open(filename, 'w') as file:
        for key, value in parameter_dict.items():
            file.write(f"{key} = {value}\n")

    return df_save


In [15]:
# Cluster parameters
dict_prelim = dict(alpha=0.01,
                   beta=0.99,
                   knn_initcluster_graph=35, # slightly larger than the bigget KNN
                   KNN_list=[15,20, 25],
                   sfs=[0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55],
                   scaling="robust",
                   bh_correction=True)

In [16]:
print(f"PART A) Starting clustering ... \n")

df_prelim = run_clustering(region_label=chunk, df_input=df_small, sf_params="parallax_scaled",
                           parameter_dict=dict_prelim, mode="prelim", output_loc=result_path, setup_func="ICRS")

PART A) Starting clustering ... 

-- Current run with KNN = 15 -- 

Creating k-d trees of resampled data sets...
Performing clustering for scale factor 0.1...
Performing gradient ascend using a 15-NN density estimation.
Updated significance threshold: 5.00e-02
Creating k-d trees of resampled data sets...
Performing clustering for scale factor 0.15...
Performing gradient ascend using a 15-NN density estimation.
Updated significance threshold: 5.00e-02
Creating k-d trees of resampled data sets...
Performing clustering for scale factor 0.2...
Performing gradient ascend using a 15-NN density estimation.
Updated significance threshold: 5.00e-02
Creating k-d trees of resampled data sets...
Performing clustering for scale factor 0.25...
Performing gradient ascend using a 15-NN density estimation.
Updated significance threshold: 5.00e-02
Creating k-d trees of resampled data sets...
Performing clustering for scale factor 0.3...
Performing gradient ascend using a 15-NN density estimation.
Update

In [17]:
d =df_prelim.dropna()
d = d[d.parallax >= 0]

print(len(df_prelim))
print(len(d))

d.rename(columns={"Plx":'parallax'}, inplace=True)
d["radial_velocity"] = pd.NA
d["D"] = 1000/d["parallax"]

c1 = d[d["rsc"] == 0]
c1.D.median()

507
507


381.38825324180016

### B) Simulate clusters and determine scale factors

We now want to use the preliminary solution (mostly cluster cores) to determine more appropriate scaling factors for the clustering algorithm. We calculate them and save them in a text file. It is also possible to add a couple of small, artificial clusters to make the algorithm more sensitive for small clusters.

In [19]:

print(f"PART B) Simulating clusters ... \n")

stds = calculate_std_devs(input_df=d, SigMA_dict=dict_prelim, sampling_data= error_sampling_df, n_artificial = 0, sample_radius= 2, output_path = result_path, plot_figs = False)

# save scaling factors in Data directory
directory = "../../Data/Scale_factors"
filename = f"sfs_FN_{chunk}.txt"

# Open the file in write mode ('w')
with open(f"{directory}/{filename}", 'w') as file:
    for i, label in enumerate(["ra", "dec", "parallax", "pmra", "pmdec"]):
        print(f"{label}:", np.min(stds[i, :]), np.mean(stds[i, :]), np.max(stds[i, :]), file=file)

PART B) Simulating clusters ... 

380.7199228135706
380.7199228135706
380.7199228135706
404.58926864122793
404.58926864122793
404.58926864122793


/opt/anaconda3/envs/Sigma_Orion/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



### C) Cluster with new SF

We apply the new scaling factors to the clustering algorithm. You can play with the number of scaling factors - more factors = more computing time, but hopefully also a more reliable result, as the parameter space is sampled more densely.

In [20]:
# determine the number of SF to draw using lhc_lloyd sampling
num_sf = 30

In [21]:
# draw number of scale factors
sfs, means = lhc_lloyd('../../Data/Scale_factors/' + f"sfs_FN_{chunk}.txt", num_sf)

# determine means for clusterer initialization
scale_factor_means = {'pos': {'features': ['ra', 'dec', 'parallax'], 'factor': list(means[:3])},
                      'vel': {'features': ['pmra', 'pmdec'], 'factor': list(means[3:])}}

# dict for final clustering
dict_final = dict(alpha=0.01,
                  beta=0.99,
                  knn_initcluster_graph=35,
                  KNN_list=[20, 25, 30],
                  sfs=sfs,
                  scaling=None,
                  bh_correction=False)


df_fin = run_clustering(region_label=chunk, df_input=df_chunk,
                                sf_params=["ra", "dec", "parallax", "pmra", "pmdec"],
                                parameter_dict=dict_final, mode="final", output_loc=result_path,
                                column_means=scale_factor_means)

ValueError: Bounds are not consistent 'l_bounds' < 'u_bounds'

## Output

The code produces a lot of output plots (html). They can be split into the overall results (OCC), which show three different noise removal methods:
- rfs (less strict, sometimes spurious clusters)
- rsc (more strict, default solution for me)
- simple (between rfs and rsc)

It also produces plots for the individual solutions for the different KNN.